# Data Processing

Process data into consistent format for hugging face training script.
https://github.com/huggingface/transformers/tree/master/examples/pytorch/summarization

In [39]:
!pip install sagemaker transformers nltk --upgrade

In [41]:
import json
from tqdm import tqdm

In [42]:
split = 'test'
data_path = '/home/ec2-user/SageMaker/QMSum/data/ALL/jsonl/' + split + '.jsonl'
data = []
with open(data_path) as f:
    for line in f:
        data.append(json.loads(line))
n_meetings = len(data)
print('Total {} meetings in the {} set.'.format(n_meetings, split))

Total 35 meetings in the test set.


In [43]:
from nltk import word_tokenize

def tokenize(sent):
    tokens = ' '.join(word_tokenize(sent.lower()))
    return tokens

In [44]:
# filter some noises caused by speech recognition
def clean_data(text):
    text = text.replace('{ vocalsound } ', '')
    text = text.replace('{ disfmarker } ', '')
    text = text.replace('a_m_i_', 'ami')
    text = text.replace('l_c_d_', 'lcd')
    text = text.replace('p_m_s', 'pms')
    text = text.replace('t_v_', 'tv')
    text = text.replace('{ pause } ', '')
    text = text.replace('{ nonvocalsound } ', '')
    text = text.replace('{ gap } ', '')
    return text

In [45]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [46]:
# process data for BART
# the input of the model here is the entire content of the meeting
processed_data = []
for i in tqdm(range(len(data))):
    # get meeting content
    text = []
    for k in range(len(data[i]['meeting_transcripts'])):
        cur_turn = data[i]['meeting_transcripts'][k]['speaker'].lower() + ': '
        cur_turn = cur_turn + tokenize(data[i]['meeting_transcripts'][k]['content'])
        text.append(cur_turn)
    text = ' '.join(text)
    text = clean_data(text)
    
    summary = data[i]['general_query_list'][0]['answer']
    summary = tokenize(summary)
    
    processed_data.append({'text': text, 'summary': summary})
        
print('Total {} query-summary pairs in the {} set'.format(len(processed_data), split))
print(processed_data[2])
with open('data_' + split + '.jsonl', 'w') as f:
    for i in range(len(processed_data)):
        print(json.dumps(processed_data[i]), file=f)

100%|██████████| 35/35 [00:04<00:00,  7.41it/s]

Total 35 query-summary pairs in the test set
{'text': "marketing: okay . project manager: 'kay so we 'll try to zip through this , since we 're short on time . welcome everybody . um hope your sessions went well . um so this is our functional design meeting , we 're going to consider um user needs , technical effects , and the working design of our remote control . um i 've been taking meetings on the minute minutes on the meetings , and i 'll be putting them in the shared documents folder so if there 's anything you need to refer to you can find them in there . um i i 'll get the ones up for next time , um they 're not finished yet . right . um so can we have updates from everyone from what you 've worked on just kind of a quick summary of anything interesting that you 'd like to share or discuss in this marketing: and i can start if you want . mm . project manager: sure . marketing: is there an order ? no . project manager: hm ? marketing: we have n't decided on an order . project ma